In [2]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
import requests
import pandas as pd
import time
import os

# ✅ Korrekte CoinGecko-IDs für die gewünschten Kryptowährungen
CRYPTO_LIST = {
    "Solana": "solana",
    "Avalanche": "avalanche-2",
    "Polkadot": "polkadot",
    "Near Protocol": "near",
    "Polygon": "matic-network",
    "XRP": "ripple",
    "Cardano": "cardano"
}

# 🕒 Zeitraum für die letzten 90 Tage
DAYS = 90

# 📌 Lokaler Speicherpfad (Google Drive Sync-Ordner)
DRIVE_PATH = "G:/Meine Ablage/reddit/"
OUTPUT_FILE = os.path.join(DRIVE_PATH, "crypto_prices.csv")

# 📌 Liste zur Speicherung der Kursdaten
all_prices = []

# 📌 API-Abfrage für jede Kryptowährung mit besserem Rate-Limit-Handling
for crypto_name, crypto_id in CRYPTO_LIST.items():
    print(f"📡 Abrufen von Kursdaten für {crypto_name}...")

    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart?vs_currency=usd&days={DAYS}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        prices = data["prices"]  # Liste von [timestamp, price]

        for price_data in prices:
            all_prices.append({
                "date": price_data[0],  # Speichert den Timestamp erstmal als Zahl
                "crypto": crypto_name,
                "price": price_data[1]
            })

        print(f"✅ Erfolgreich abgerufen: {crypto_name}")

    elif response.status_code == 429:
        print(f"⚠️ API-Limit erreicht für {crypto_name}. Warte 60 Sekunden...")
        time.sleep(60)  # ⏳ Wartezeit für Rate-Limit
        continue  # Überspringe und versuche später

    else:
        print(f"⚠️ Fehler beim Abrufen der API für {crypto_name}: {response.status_code}")

    # ⏳ Standardpause von 10 Sekunden zwischen den Requests
    time.sleep(10)

# 📌 DataFrame erstellen
df_prices = pd.DataFrame(all_prices)

# ✅ Datum korrekt formatieren
df_prices["date"] = pd.to_datetime(df_prices["date"], unit="ms").dt.date  # Umwandlung von Timestamp zu Date

# ✅ Daten in Google Drive speichern
df_prices.to_csv(OUTPUT_FILE, sep="|", encoding="utf-8-sig", index=False)

print(f"✅ Kursdaten für die letzten 90 Tage gespeichert: {OUTPUT_FILE}")


📡 Abrufen von Kursdaten für Solana...
✅ Erfolgreich abgerufen: Solana
📡 Abrufen von Kursdaten für Avalanche...
✅ Erfolgreich abgerufen: Avalanche
📡 Abrufen von Kursdaten für Polkadot...
✅ Erfolgreich abgerufen: Polkadot
📡 Abrufen von Kursdaten für Near Protocol...
✅ Erfolgreich abgerufen: Near Protocol
📡 Abrufen von Kursdaten für Polygon...
✅ Erfolgreich abgerufen: Polygon
📡 Abrufen von Kursdaten für XRP...
✅ Erfolgreich abgerufen: XRP
📡 Abrufen von Kursdaten für Cardano...
✅ Erfolgreich abgerufen: Cardano
✅ Kursdaten für die letzten 90 Tage gespeichert: G:/Meine Ablage/reddit/crypto_prices.csv


In [5]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15106 entries, 0 to 15105
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    15106 non-null  object 
 1   crypto  15106 non-null  object 
 2   price   15106 non-null  float64
dtypes: float64(1), object(2)
memory usage: 354.2+ KB
